In [1]:
import pandas as pd
import numpy as np
import zipfile
import requests
import os
import glob
from os import chdir, getcwd, listdir
from io import BytesIO

from pyspark.sql.types import *
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession

#-------------------------------
# bibliotecas webscraping
import urllib3
from bs4 import BeautifulSoup

urllib3.disable_warnings()

spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

url = 'https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/indicadores-de-fluxo-da-educacao-superior/2010-2019'
folder = 'indicadores_fluxo_educacao_superior'
path = '/tmp/'

In [ ]:
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Criando diretório para armazenar o conteúdo do INEP ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
os.makedirs(folder, exist_ok=True)

print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Efetuando busca webscraping da URL de download do arquivo ZIP ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
conexao = urllib3.PoolManager()
retorno = conexao.request('GET', url)

pagina = BeautifulSoup(retorno.data,"html.parser")

dado = []
for link in pagina.find_all('a',class_ = 'external-link'):
    dado.append(link.get('href'))

url_download = str(dado[0])

print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Efetuando download do arquivo Zip ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
# Desabilitar temporariamente a verificação SSL
response = requests.get(url_download, verify=False)
filebytes = BytesIO(response.content)

print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Descompactando arquivo Zip no Lake ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

myzip = zipfile.ZipFile(filebytes)
myzip.extractall(path + folder)
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Descompactação efetuada ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

chdir(path + folder)
print(getcwd())
for c in listdir():
    print(c)

In [ ]:
# arquivo = url_download.split('/')[-1]
# print(arquivo)
# type(arquivo)

In [ ]:
# pip install openpyxl

In [ ]:
# targetPattern = r"/tmp/indicadores_fluxo_educacao_superior/*.xlsx"
# file = str(glob.glob(targetPattern)).replace("['","").replace("']","")
# print(file)

In [2]:
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Efetuando Busca do nome do Arquivo XLSX ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

targetPattern = r"{path}{folder}/*.xlsx".format(path=path, folder=folder)
file = str(glob.glob(targetPattern)).replace("['","").replace("']","")
print("Arquivo:", file)

print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Carregando Dataframe com os dados do Excel ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

df = pd.read_excel(file, usecols= 'A:AE', skiprows= lambda x: x < 8 or x > 259238, sheet_name='INDICADORES_TRAJETORIA' )

============ ============ ============ ============ ============ ============ ============ ============ ============
============ Efetuando Busca do nome do Arquivo XLSX ============
============ ============ ============ ============ ============ ============ ============ ============ ============
Arquivo: /tmp/indicadores_fluxo_educacao_superior/indicadores_trajetoria_educacao_superior_2010_2019.xlsx
============ ============ ============ ============ ============ ============ ============ ============ ============
============ Carregando Dataframe com os dados do Excel ============
============ ============ ============ ============ ============ ============ ============ ============ ============


In [ ]:
df.head()

In [ ]:
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Convertendo Dataframe para SPARK ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

sparkDF=spark.createDataFrame(df) 
# sparkDF.printSchema()
# sparkDF.show()

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
sparkDF.show()

In [ ]:
import pyspark.sql.functions as fn

In [3]:
df.isnull().sum()

CO_IES                             0
NO_IES                             0
TP_CATEGORIA_ADMINISTRATIVA        0
TP_ORGANIZACAO_ACADEMICA           0
CO_CURSO                           0
NO_CURSO                           0
CO_REGIAO                       6970
CO_UF                           6970
CO_MUNICIPIO                    6970
TP_GRAU_ACADEMICO                  0
TP_MODALIDADE_ENSINO               0
CO_CINE_ROTULO                     0
NO_CINE_ROTULO                     0
CO_CINE_AREA_GERAL                 0
NO_CINE_AREA_GERAL                 0
NU_ANO_INGRESSO                    0
NU_ANO_REFERENCIA                  0
NU_PRAZO_INTEGRALIZACAO            0
NU_ANO_INTEGRALIZACAO              0
NU_PRAZO_ACOMPANHAMENTO            0
NU_ANO_MAXIMO_ACOMPANHAMENTO       0
QT_INGRESSANTE                     0
QT_PERMANENCIA                     0
QT_CONCLUINTE                      0
QT_DESISTENCIA                     0
QT_FALECIDO                        0
TAP                                0
T

In [12]:
# print(df['CO_UF'].unique())
df['CO_UF'].isna().sum()

6970

In [ ]:
DFAjusteTipoDado = (sparkDF
    .select(fn.col('CO_IES').cast('bigint').alias('CO_IES'),
            fn.col('NO_IES').cast('string').alias('NO_IES'),
            fn.col('TP_CATEGORIA_ADMINISTRATIVA').cast('int').alias('TP_CATEGORIA_ADMINISTRATIVA'),
            fn.col('TP_ORGANIZACAO_ACADEMICA').cast('int').alias('TP_ORGANIZACAO_ACADEMICA'),
            fn.col('CO_CURSO').cast('bigint').alias('CO_CURSO'),
            fn.col('NO_CURSO').cast('string').alias('NO_CURSO'),
            fn.when(fn.trim(fn.col('CO_REGIAO')) == 'NaN', None).otherwise(fn.col('CO_REGIAO')).cast('int').alias('CO_REGIAO'),
            fn.when(fn.trim(fn.col('CO_UF')) == 'NaN', None).otherwise(fn.col('CO_UF')).cast('int').alias('CO_UF'),
            fn.when(fn.trim(fn.col('CO_MUNICIPIO')) == 'NaN', None).otherwise(fn.col('CO_MUNICIPIO')).cast('bigint').alias('CO_MUNICIPIO'),
            fn.col('TP_GRAU_ACADEMICO').cast('int').alias('TP_GRAU_ACADEMICO'),
            fn.col('TP_MODALIDADE_ENSINO').cast('int').alias('TP_MODALIDADE_ENSINO'),
            fn.col('CO_CINE_ROTULO').cast('string').alias('CO_CINE_ROTULO'),
            fn.col('NO_CINE_ROTULO').cast('string').alias('NO_CINE_ROTULO'),
            fn.col('CO_CINE_AREA_GERAL').cast('int').alias('CO_CINE_AREA_GERAL'),
            fn.col('NO_CINE_AREA_GERAL').cast('string').alias('NO_CINE_AREA_GERAL'),
            fn.col('NU_ANO_INGRESSO').cast('int').alias('NU_ANO_INGRESSO'),
            fn.col('NU_ANO_REFERENCIA').cast('int').alias('NU_ANO_REFERENCIA'),
            fn.col('NU_PRAZO_INTEGRALIZACAO').cast('int').alias('NU_PRAZO_INTEGRALIZACAO'),
            fn.col('NU_ANO_INTEGRALIZACAO').cast('int').alias('NU_ANO_INTEGRALIZACAO'),
            fn.col('NU_PRAZO_ACOMPANHAMENTO').cast('int').alias('NU_PRAZO_ACOMPANHAMENTO'),
            fn.col('NU_ANO_MAXIMO_ACOMPANHAMENTO').cast('int').alias('NU_ANO_MAXIMO_ACOMPANHAMENTO'),
            fn.col('QT_INGRESSANTE').cast('bigint').alias('QT_INGRESSANTE'),
            fn.col('QT_PERMANENCIA').cast('bigint').alias('QT_PERMANENCIA'),
            fn.col('QT_CONCLUINTE').cast('bigint').alias('QT_CONCLUINTE'),
            fn.col('QT_DESISTENCIA').cast('bigint').alias('QT_DESISTENCIA'),
            fn.col('QT_FALECIDO').cast('bigint').alias('QT_FALECIDO'),
            fn.col('TAP').cast('double').alias('TAP'),
            fn.col('TCA').cast('double').alias('TCA'),
            fn.col('TDA').cast('double').alias('TDA'),
            fn.col('TCAN').cast('double').alias('TCAN'),
            fn.col('TADA').cast('double').alias('TADA')
           )
)



In [ ]:
DFAjusteTipoDado.groupBy(fn.col('CO_MUNICIPIO').isNull()).agg(fn.count('*')).show(100, False)

In [ ]:
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Gravando Dados na Camada BRONZE  ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

(sparkDF
 .write
 .format('parquet')
 .mode('overwrite')
 .save('s3a://bronze/teste/indicadores_fluxo_educacao_superior')
 )

In [ ]:
# sparkDF.printSchema()

In [ ]:
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Carregando Dataframe a partir do Arquivo Parquet ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

dfIndicador = (spark.read.format('parquet')
             .load('s3a://landing/indicadores_fluxo_educacao_superior'))

In [ ]:
print('============ ============ ============ ============ ============ ============ ============ ============ ============')
print('============ Evidência de Gravação ============')
print('============ ============ ============ ============ ============ ============ ============ ============ ============')

dfIndicador.show(5, False)

In [ ]:
print("Total de registros Carregados no Arquivo Parquet:", dfIndicador.count())